In [1]:
import pandas as pd
import glob
import sys
import json
import csv

In [2]:
df_seqs = pd.read_csv("data/sequence_data/AllMnaullyImgLevelLabeledSeqsWith10SBPPlusAllSeqsWithOneAnn2beCleaned", 
                      sep=":", 
                      names=['img', 'label'])
seq_size = 10

## get sequences with len 10 in flat format

In [39]:
sys.stdout = open("seqsWLen10WOneAnnFlat", "w")
res_df = pd.DataFrame()
seq_counts = 0
for path in glob.glob("new_train_noBack_annotations/" + "*.png"):
    img_name = path.split("/")[-1][:-4]
    match = df_seqs[df_seqs['img'].str.contains(img_name)]
    if match.shape[0] > 0:
        match = match.reset_index(drop = True)
        labels = match.label.unique()
        for label in labels:
            seq_match = df_seqs[df_seqs['label'].str.contains(label)]
            seq_match = seq_match.reset_index(drop = True)
            if seq_match.shape[0] >= seq_size - 1:
                print("/usb/seq_data_for_mit_code/{}.JPG: new_train_noBack_annotations/{}.png".
                     format(img_name, img_name))
                res = seq_match.sample(seq_size - 1).reset_index(drop = True)
                for row in res.iterrows():
                    print("/usb/seq_data_for_mit_code/{}: new_train_noBack_annotations/{}.png".
                          format(row[1].img.split('/')[-1].replace('.icon',''), img_name))
                seq_counts += 1



## get sequences with len 10 in odgt format

In [5]:
with open("seqsWLen10WOneAnn.odgt", "w") as fw:
    seq_counts = 0
    for path in glob.glob("new_train_noBack_annotations/" + "*.png"):
        img_name = path.split("/")[-1][:-4]
        match = df_seqs[df_seqs['img'].str.contains(img_name)]
        if match.shape[0] > 0:
            match = match.reset_index(drop = True)
            labels = match.label.unique()
            for label in labels:
                seq_match = df_seqs[df_seqs['label'].str.contains(label)]
                seq_match = seq_match.reset_index(drop = True)
                if seq_match.shape[0] >= seq_size - 1:
                    res = seq_match.sample(seq_size - 1).reset_index(drop = True)
                    img_names = ["/usb/seq_data_for_mit_code/" + img_name + '.JPG']
                    seg_names = ["new_train_noBack_annotations/" + img_name + '.png']
                    for row in res.iterrows():
                        img_names.append("/usb/seq_data_for_mit_code/" + row[1].img.split('/')[-1].replace('.icon',''))
                        seg_names.append("new_train_noBack_annotations/" + img_name + '.png')
                    new_line = {}
                    new_line["fpath_img"], new_line["fpath_segm"], new_line["width"], new_line["height"] = \
                                img_names, seg_names, 600, 400
                    json.dump(new_line, fw)
                    fw.write('\n')
                    seq_counts += 1


## get mit format data for sequences with size 2 to 10

In [3]:
def format_seq(inputfile, length):
    with open(inputfile, "r") as f_r, open("cct_seq_" + str(length), 'w') as f_cct, open("mit_seq_" + str(length), 'w') as f_mit:
        sequence_data = f_r.readlines()
        write = csv.writer(f_cct, delimiter = ' ')
        for line in sequence_data:
            new_line = json.loads(line)
            new_line['fpath_img'] = new_line['fpath_img'][:length]
            new_line['fpath_segm'] = new_line['fpath_segm'][:length]
            json.dump(new_line, f_mit)
            f_mit.write('\n')
            
            write.writerow(new_line['fpath_img'][:length])
            

In [4]:
for i in range(2, 11):
    format_seq("data/seqsWLen10WOneAnn.odgt", i)

In [15]:
line['fpath_img'][:2]

['/usb/seq_data_for_mit_code/cac01124.06.JPG',
 '/usb/seq_data_for_mit_code/cac01126.05.JPG']

In [30]:
len(sequence_data)

1216